# Visualization: Trading Session

In [10]:
import pandas as pd
import numpy as np

import altair as alt
import seaborn as sns

### 1. Define parameters and Load model

In [11]:
from trading_bot.agent import Agent

model_name = 'QQQ1m_20'
test_stock = 'data/QQQ_1m_test.csv'
window_size = 10
debug = True

agent = Agent(window_size, pretrained=True, model_name=model_name)

### 2. Load test data

In [15]:
# read csv into dataframe
df = pd.read_csv(test_stock)
# filter out the desired features
#df = df[['Date', 'Adj Close']]
#df = df.rename(columns={'Adj Close': 'actual', 'Date': 'date'})

## minute
df = df[['Datetime', 'Adj Close']] # 4 minute data
df = df.rename(columns={'Adj Close': 'actual', 'Datetime': 'date'})
# rename feature column names
# convert dates from object to DateTime type
dates = df['date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df['date'] = dates

df.head()

,date,actual
0,2020-09-04 11:29:00-04:00,275.359985
1,2020-09-04 11:30:00-04:00,276.130005
2,2020-09-04 11:31:00-04:00,275.769989
3,2020-09-04 11:32:00-04:00,275.910004
4,2020-09-04 11:33:00-04:00,276.209991


### 3. Running Eval

In [16]:
import logging
import coloredlogs

from trading_bot.utils import show_eval_result, switch_k_backend_device, get_stock_data
from trading_bot.methods import evaluate_model

coloredlogs.install(level='DEBUG')
switch_k_backend_device()

test_data = get_stock_data(test_stock)
initial_offset = test_data["Adj Close"][1] - test_data["Adj Close"][0]

test_result, history = evaluate_model(agent, test_data, window_size, debug)
show_eval_result(model_name, test_result, initial_offset)

2020-09-07 14:47:24 justin-desktop-ubuntu root[70975] DEBUG switching to TensorFlow for CPU
2020-09-07 14:47:24 justin-desktop-ubuntu root[70975] DEBUG Buy at: $275.36
2020-09-07 14:47:24 justin-desktop-ubuntu root[70975] DEBUG Sell at: $276.13 | Position: +$0.77
2020-09-07 14:47:24 justin-desktop-ubuntu root[70975] DEBUG Buy at: $275.77
2020-09-07 14:47:24 justin-desktop-ubuntu root[70975] DEBUG Buy at: $275.91
2020-09-07 14:47:24 justin-desktop-ubuntu root[70975] DEBUG Buy at: $276.21
2020-09-07 14:47:24 justin-desktop-ubuntu root[70975] DEBUG Buy at: $277.25
2020-09-07 14:47:24 justin-desktop-ubuntu root[70975] DEBUG Sell at: $277.32 | Position: +$1.55
2020-09-07 14:47:24 justin-desktop-ubuntu root[70975] DEBUG Buy at: $277.79
2020-09-07 14:47:24 justin-desktop-ubuntu root[70975] DEBUG Buy at: $278.00
2020-09-07 14:47:24 justin-desktop-ubuntu root[70975] DEBUG Buy at: $278.35
2020-09-07 14:47:24 justin-desktop-ubuntu root[70975] DEBUG Sell at: $278.45 | Position: +$2.54
2020-09-07 1

2020-09-07 14:47:24 justin-desktop-ubuntu root[70975] DEBUG Buy at: $279.69
2020-09-07 14:47:24 justin-desktop-ubuntu root[70975] DEBUG Buy at: $279.87
2020-09-07 14:47:24 justin-desktop-ubuntu root[70975] DEBUG Buy at: $279.63
2020-09-07 14:47:24 justin-desktop-ubuntu root[70975] DEBUG Buy at: $279.70
2020-09-07 14:47:24 justin-desktop-ubuntu root[70975] DEBUG Buy at: $280.12
2020-09-07 14:47:24 justin-desktop-ubuntu root[70975] DEBUG Buy at: $279.82
2020-09-07 14:47:24 justin-desktop-ubuntu root[70975] DEBUG Buy at: $279.71
2020-09-07 14:47:24 justin-desktop-ubuntu root[70975] DEBUG Buy at: $279.55
2020-09-07 14:47:24 justin-desktop-ubuntu root[70975] DEBUG Buy at: $279.49
2020-09-07 14:47:24 justin-desktop-ubuntu root[70975] DEBUG Buy at: $279.78
2020-09-07 14:47:24 justin-desktop-ubuntu root[70975] DEBUG Buy at: $279.33
2020-09-07 14:47:24 justin-desktop-ubuntu root[70975] DEBUG Buy at: $279.52
2020-09-07 14:47:24 justin-desktop-ubuntu root[70975] DEBUG Buy at: $280.12
2020-09-07 1

Final decision:  BUY  at  $283.76


In [17]:
#+$294.03

### 4. Visualize

In [18]:
def visualize(df, history, title="trading session"):
    # add history to dataframe
    position = [history[0][0]] + [x[0] for x in history]
    actions = ['HOLD'] + [x[1] for x in history]
    df['position'] = position
    df['action'] = actions
    
    # specify y-axis scale for stock prices
    scale = alt.Scale(domain=(min(min(df['actual']), min(df['position'])) - 50, max(max(df['actual']), max(df['position'])) + 50), clamp=True)
    
    # plot a line chart for stock positions
    actual = alt.Chart(df).mark_line(
        color='green',
        opacity=0.5
    ).encode(
        x='date:T',
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale)
    ).interactive(
        bind_y=False
    )
    
    # plot the BUY and SELL actions as points
    points = alt.Chart(df).transform_filter(
        alt.datum.action != 'HOLD'
    ).mark_point(
        filled=True
    ).encode(
        x=alt.X('date:T', axis=alt.Axis(title='Date')),
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale),
        color='action'
    ).interactive(bind_y=False)

    # merge the two charts
    chart = alt.layer(actual, points, title=title).properties(height=300, width=1000)
    
    return chart

In [19]:
chart = visualize(df, history, title=test_stock)
chart

alt.LayerChart(...)